## YAAD Dataset Import and Feature Selection

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

# Packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns
#import plotly.express as px
import mne
import matplotlib.pyplot as plt
import numpy as np
from numpy import array
import scipy.io
import scipy.signal
import pandas as pd
import neurokit2 as nk
import numpy as np
import pandas as pd
import os

### Preprocess and HRV extraction

In [ ]:


from acsef_hrv_extraction import hrv_extraction

hrv_df = pd.DataFrame()

dir = "/Users/nivinattudurai/Downloads/YAAD 1/ECG_GSR_Emotions/Raw Data/Multimodal/ECG/"

def by_numbers(s): 
    return int(''.join(char for char in s if char.isdigit()))

files = sorted(os.listdir(dir), key=by_numbers)

for filename in files:
    hrv = hrv_extraction(dir+filename)
    hrv_df = pd.concat([hrv_df, hrv])
    print(hrv_df.shape)

### NaN heatmap

In [ ]:
plt.figure(figsize=(21,7))
sns.heatmap(hrv_df.isna().values, cmap="crest", xticklabels=hrv_df.columns)
plt.title('NaN heatmap')
plt.show()

### Drop 100% missing value columns

In [2]:
missing_count = hrv_df.isna().sum()
missing_df=(pd.concat([missing_count.rename('Missing count'),
                      missing_count.div(len(hrv_df)).rename('Missing percentage')], axis=1)).loc[missing_count.ne(0)]

print("Number of columns with missing data (null values) in train daaset = ", len(missing_df) )
print()
print("Missing data stats :")
missing_df.sort_values(by=['Missing percentage'], ascending=False)

Number of columns with missing data (null values) in train daaset =  16

Missing data stats :


,Missing count,Missing percentage
HRV_SDANN1,252,1.000000
HRV_SDNNI1,252,1.000000
HRV_SDANN2,252,1.000000
HRV_SDNNI2,252,1.000000
HRV_SDANN5,252,1.000000
HRV_SDNNI5,252,1.000000
HRV_ULF,252,1.000000
HRV_VLF,252,1.000000
HRV_LF,252,1.000000
HRV_LFHF,252,1.000000


In [3]:
drop_cols = ['HRV_SDANN1', 'HRV_SDNNI1', 'HRV_SDANN2', 'HRV_SDNNI2', 'HRV_SDANN5', 'HRV_SDNNI5', 'HRV_ULF', 'HRV_VLF', 'HRV_LF', 'HRV_LFHF', 'HRV_LFn']
hrv_df.drop(drop_cols, inplace=True, axis=1)

### Replace missing value columns with median

In [4]:
all_num_cols = set(list(hrv_df.select_dtypes('number').columns))
replace_cols = set(list(missing_df.index)) - set(drop_cols)
replace_num_cols = list(all_num_cols.intersection(replace_cols))


replace_vals_hrv = {}
for col in replace_num_cols:
    replace_vals_hrv[col] = hrv_df[col].median()


In [5]:
hrv_df = hrv_df.fillna(value=replace_vals_hrv)


### Scaling data

In [8]:
# Scaling the data 
scaler = MinMaxScaler()
hrv_df[replace_num_cols] = scaler.fit_transform(hrv_df[replace_num_cols])

### Classification Label Matrix and Encoding

In [9]:
df2 = pd.read_csv("/Users/nivinattudurai/Downloads/YAAD 1/ECG_GSR_Emotions/Self-Annotation Labels/Self-annotation Multimodal copy.csv")
df3 = pd.DataFrame(df2)

replacements = {'VeryHigh': 5, 'High': 4, 'Moderate': 3, 'Low': 2, 'VeryLow': 1}


df3[["Happy", "Sad", "Fear", "Anger", "Neutral", "Disgust", "Surprised"]] = df3[["Happy", "Sad", "Fear", "Anger", "Neutral", "Disgust", "Surprised"]].replace(replacements)
df3

r1 = df3[["Happy", "Sad", "Fear", "Anger", "Neutral", "Disgust", "Surprised"]]



In [ ]:
r2 = r1
r2.loc[r2["Happy"] >= 3, 'Happy'] = 0
r2.loc[r2["Happy"] != 0, 'Happy'] = -1
r2.loc[r2["Sad"] >= 3, 'Sad'] = 0
r2.loc[r2["Sad"] != 0, 'Sad'] = -1
r2.loc[r2["Fear"] >= 3, 'Fear'] = 0
r2.loc[r2["Fear"] != 0, 'Fear'] = -1
r2.loc[r2["Anger"] >= 3, 'Anger'] = 0
r2.loc[r2["Anger"] != 0, 'Anger'] = -1
r2.loc[r2["Neutral"] >= 3, 'Neutral'] = 0
r2.loc[r2["Neutral"] != 0, 'Neutral'] = -1
r2.loc[r2["Disgust"] >= 3, 'Disgust'] = 0
r2.loc[r2["Disgust"] != 0, 'Disgust'] = -1
r2.loc[r2["Surprised"] >= 3, 'Surprised'] = 0
r2.loc[r2["Surprised"] != 0, 'Surprised'] = -1



## Machine Learning

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestClassifier


In [12]:
from sklearn.decomposition import PCA

X = hrv_df
y = r2

X = X.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)



#X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.80, random_state=1,stratify=y)


### Apply PCA

In [16]:
pca = PCA()
pca_X = pca.fit_transform(X)
pca_X = pd.DataFrame(pca_X)

In [4]:
#all_indices = list(range(len(X)))
#train_indices, test_indices = train_test_split(all_indices, test_size=0.2)

#X_train = array(pca_X.iloc[train_indices])
#X_test = array(pca_X.iloc[test_indices])
#Y_train = array(y.iloc[train_indices, :])
#Y_test = array(y.iloc[test_indices, :])

#X_test = X_test.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)






In [5]:
#forest = RandomForestClassifier(random_state=1)
#multi_target_forest = MultiOutputClassifier(forest, n_jobs=2)
#model = multi_target_forest.fit(X_train, Y_train)
#preds1 = model.predict(X_test)
#model.score(X_test, Y_test)


### Neural Network

In [13]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

2024-03-09 14:59:07.881887: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Build model framework

In [14]:
def baseline_model():
 # create model
 model = Sequential()
 model.add(Dense(5, input_dim=69, activation='relu'))
 model.add(Dense(2, activation='relu'))
 model.add(Dense(7, activation='softmax'))
 # Compile model
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 return model

In [17]:
X_a = array(pca_X)
y_a = array(y)

In [ ]:
#encoder = LabelEncoder()
#encoder.fit(y_a)
#encoded_Y = encoder.transform(y_a)
# convert integers to dummy variables (i.e. one hot encoded)
#dummy_y = np_utils.to_categorical(encoded_Y)

In [18]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=5, shuffle=True)

/Users/nivinattudurai/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


### Final accuracy

In [19]:

results = cross_val_score(estimator, X_a, y_a, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

2024-03-09 15:01:43.638291: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Baseline: 74.40% (38.75%)
